## Step 1: Upload list of addressses.

In [50]:
'''
A list of all token holders can be found at the dune dahsboard: https://dune.com/tahasaeed/beanstalk-research
An update tot he dune dashboard allos Dune users to download tables. Click into the query results view for the 
table 'BeanStalk] - Account Holdings'. Below the query, on the right hand side of the page, you'll see a "Export to CSV button."
'''
import pandas as pd
holder_df = pd.read_csv('/Users/tahasaeed/Documents/GreenQuant Analytics/Beanstalk - Analysis - Holders data.csv')

# select just list of addresses 
holder = holder_df['address']

In [18]:
''' Load subgraph query funtions '''
import requests
import pandas as pd
import sys
from collections import OrderedDict

BEANSTALK_SUBGRAPH = "https://graph.node.bean.money/subgraphs/name/beanstalk"
SUBGRAPH = "https://gateway.thegraph.com/api/fe672ef9fcdfb617c4d7755f36a31131/subgraphs/id/0x925753106fcdb6d2f30c3db295328a0a1c5fd1d1-0"

token_addrs = OrderedDict(dict(
    bean='0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab'.lower(), 
    bean_3crv='0xc9C32cd16Bf7eFB85Ff14e0c8603cc90F6F2eE49'.lower(), 
    ur_bean='0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449'.lower(), 
    ur_bean_3crv='0x1BEA3CcD22F4EBd3d37d731BA31Eeca95713716D'.lower(), 
))

## this query gets  deposits per seasion, token 
def farmerQuery(id):
    return """
    {
    farmer(id: "ID") {
        deposits(orderBy: season, where: {amount_gt: 0}) {
          id
          season
          token
          amount
          withdrawnAmount
        }
     }
    }
    """.replace("ID", str(id))

def run_cujo_query(query):
  request = requests.post(SUBGRAPH,'',json={'query': query})
  
  if request.status_code == 200:
      return request.json()
  else:
      raise Exception('Query failed. return code is {}.      {}'.format(
              request.status_code, request
          )
      )

def query_farmer_deposits(id):
  query = run_cujo_query(farmerQuery(id.lower()))

  if list(query.keys())[0] == "errors":
    print("query_farmer_deposits: Error querying")

  if query['data']['farmer'] == None:
    print("query_farmer_deposits: No deposits found for account " + id)
    
  else:
    data = query['data']['farmer']['deposits']
    df = pd.DataFrame(data)
  
    return df

def getDeposits(account):
  data = query_farmer_deposits(account)
  df = pd.DataFrame(data)

  if not df.empty:
    df['amount'] = df['amount']
    df['withdrawnAmount'] = df['withdrawnAmount']

    # Adds names to token addresses
    token_addr_inv = {v: k for k, v in token_addrs.items()}
    df['token_name'] = df.token.apply(lambda v: token_addr_inv[v])
  else:
    print("Error in getDeposits: Dataframe is empty")

  return df

def get_Bean_Deposits(account):
  df = getDeposits(account)
  df_bean = df.loc[df.token_name == 'bean']

  return getSeasonsAndAmounts(df_bean)

def get_Total_Bean_Deposits(account):
  df = getDeposits(account)
  df_bean = df.loc[df.token_name == 'bean']
  totalDepositedBeans = 0

  crates, amounts = getSeasonsAndAmounts(df_bean)

  for amount in amounts:
    totalDepositedBeans += amount

  return totalDepositedBeans

def get_UrBean_Deposits(account):
  df = getDeposits(account)
  df_ur_bean = df.loc[df.token_name == 'ur_bean']

  return getSeasonsAndAmounts(df_ur_bean)

def get_Bean3CRV_Deposits(account):
  df = getDeposits(account)
  df_bean_3crv = df.loc[df.token_name == 'bean_3crv']

  return getSeasonsAndAmounts(df_bean_3crv)

def get_UrBean3CRV_Deposits(account):
  df = getDeposits(account)
  df_ur_bean_3crv = df.loc[df.token_name == 'ur_bean_3crv']

  return getSeasonsAndAmounts(df_ur_bean_3crv)

def getSeasonsAndAmounts(dataframe):
  seasons = []
  amounts = []

  dataframe = dataframe.reset_index()

  for index, row in dataframe.iterrows():
    season = row['season']
    amount = int(row['amount'])# - int(row['removedTokenAmount'])

    seasons.append(season)
    amounts.append(amount)

  return seasons, amounts

## Step 2: Process list of token holders to see how much holders have historically depostied.

In [27]:
# makes a dataframe of historical depostis
def beanstalk_deposits_dataframe(list_of_addresses):
    import pandas as pd
    
    new = pd.DataFrame()
    # for loop through the list of account ids to get each account's deposits
    for i in list_of_addresses: 
      # get deposits dataframe 
        deposits = getDeposits(i) 

      # check if the dataframe is empty 
        if deposits.empty: 
        # skip empty dataframes
            continue 
  
        # concatenate dataframes
        new = pd.concat([new, deposits])

    return new


def export(path,filename,dataframe_object):
    #save the dataframe as CSV file
    dataframe_object.to_csv(path + '/' + filename + '.csv')
    
def process_depositors_df(depositors_df):
    depositors_df['address'] = depositors_df['id'].str.split('-').str[0]
    depositors_df['amount'] = depositors_df['amount'].astype(float)/10**6
    return depositors_df

# create interactive pie graph of deposits
def pie_chart(token_df, limit, token_name): 
    address_totals = token_df.groupby('address')['amount'].sum()
    address_totals = pd.DataFrame(address_totals).sort_values(by = 'amount', ascending=False)
    address_totals = address_totals.reset_index()

    address_totals['labels'] = address_totals.apply(lambda row: row['address'] if row['amount'] > limit else 'other', axis=1)
    address_totals = address_totals.groupby('labels')['amount'].sum()
    address_totals = pd.DataFrame(address_totals).sort_values(by = 'amount', ascending=False)
    address_totals = address_totals.reset_index()

    import plotly.express as px
    fig = px.pie(address_totals, values='amount', names='labels', title= 'Deposits by Address:'+ token_name)
    return fig.show()

In [92]:
depositors_df = beanstalk_deposits_dataframe(holder)

Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x4b9184df8fa9f3fc008fcde7e7bbf7208ef5118d
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0xc42593f89d4b6647bff86fa309c72d5e93a9405c
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0xb899183a31501d3641e6a2ab210c857b8923c800
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x0000000000000000000000000000000000000000
Error in getDeposits: Dataframe is empty
q

Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x8521cfdfd0fd1ac18520fa8f262eb948d4420c8a
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x14567c85977847836c17dd609a9bd767f3f03b51
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x5179eab882e9f0822c87b24a48d6be45a4e7b7fb
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x45f0c134bdb3f8870bf76e57bb8f7729530002af
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x99a58482bd75cbab83b27ec03ca68ff489b5788f
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x8521cfdfd0fd1a

query_farmer_deposits: No deposits found for account 0x84cc646eb2891f1b5314ba6e2bb5886f1ffcae0a
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x227b5a7e3f56cba0cbd747760e43f5c408a13541
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x1111111254eeb25477b68fb85ed929f73a960582
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x00ea27d7da1c0fc27a198374004efe546f655aef
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0x9edaff6744d6f539298cddd8c0171b774be2b86d
Error in getDeposits: Dataframe is empty
Error in getDeposits: Dataframe is empty
query_farmer_deposits: No deposits found for account 0xeaa09621858830

In [22]:
#specify the path to the folder you want to download the CSV file to
path = '/Users/tahasaeed/Documents/GreenQuant Analytics'
#specify the filename
filename = 'deposits_log'
dataframe_object = 

export(path,filename,dataframe_object, holders_df)

## Process deposits data frame to add token name

In [94]:
p_depositors_df = process_depositors_df(depositors_df)

## How many users have deposits by token?

In [99]:
print(p_depositors_df['address'].nunique())
print(f"{round(p_depositors_df['address'].nunique()/holder_df['address'].nunique()*100, 2)}% of holders have deposits in the Beanstalk protocol")
pd.DataFrame(p_depositors_df.groupby('token_name')['address'].nunique()).rename(columns={'address':'count_unique_addresses'})

521
63.38% of holders have deposits in the Beanstalk protocol


,count_unique_addresses
token_name,
bean,311
bean_3crv,65
ur_bean,415
ur_bean_3crv,361


## How much has been deposited?

In [100]:
p_depositors_df.groupby('token_name')['amount'].sum()
# bean_3crv - use 10**18 as the decimal

token_name
bean            6.312505e+05
bean_3crv       1.778629e+17
ur_bean         1.298988e+07
ur_bean_3crv    5.528245e+07
Name: amount, dtype: float64

## How often has each holder has deposited a token?

In [101]:
pd.DataFrame(p_depositors_df[['token_name','address']].value_counts()).head(20)

0
token_name   address                                       
ur_bean      0xaf812e8babab3173edce179fe6fc6c2b1c482e39  79
ur_bean_3crv 0x4c180462a051ab67d8237ede2c987590df2fbbe6  58
             0xf1a621fe077e4e9ac2c0cefd9b69551db9c3f657  44
ur_bean      0x60e8b62c7da32ff62fcd4ab934b75d2d28fe7501  43
             0xacc53f19851ce116d52b730aee8772f7bd568821  34
bean         0xf05b641229bb2aa63b205ad8b423a390f7ef05a7  31
ur_bean_3crv 0xc2820f702ef0fbd8842c5ce8a4fcac5315593732  29
             0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db  29
             0x36998db3f9d958f0ebaef7b0b6bf11f3441b216f  28
ur_bean      0x561cbb53ba4d7912dbf9969759725bd79d920e2c  28
ur_bean_3crv 0xf05b641229bb2aa63b205ad8b423a390f7ef05a7  28
bean         0xe146313a9d6d6ccdd733cc15af3cf11d47e96f25  27
ur_bean_3crv 0xdb4fbc231f1d2b3b4c3d9e1602c895806fb06278  27
             0xe146313a9d6d6ccdd733cc15af3cf11d47e96f25  27
ur_bean      0xce66c6a88bd7bec215aa04fda4cf7c81055521d0  27
             0x83c9ec651027e061bcc39485c1fb369297bd428c  26
ur_bean_3crv 0xee7840dac7c3ec2ccde49517fd3952e349997ab5  25
             0x56a201b872b50bbdee0021ed4d1bb36359d291ed  25
ur_bean      0xefaa5fe183b7cbe2772fe4ac88831b7aa6c37422  25
ur_bean_3crv 0x4f8d7711d18344f86a5f27863051964d333798e2  24

## Let's get some descriptives about the deposits

In [102]:
df = p_depositors_df
descriptives = pd.DataFrame()
value_counts = []
for i in df['token_name'].unique():
    temp = df[df['token_name']== str(i)]
    temp = temp['amount'].describe().round()
    temp = temp.rename(i, inplace=True)
    descriptives = pd.concat([descriptives, temp], axis=1)
descriptives

,ur_bean,ur_bean_3crv,bean,bean_3crv
count,1391.0,1770.0,1084.0,1.520000e+02
mean,9339.0,31233.0,582.0,1.170151e+15
std,42276.0,95955.0,3886.0,3.139966e+15
min,0.0,0.0,0.0,3.951399e+11
25%,8.0,2004.0,2.0,9.455028e+13
50%,368.0,6269.0,15.0,3.671451e+14
75%,2337.0,22137.0,63.0,8.680768e+14
max,921951.0,1932137.0,53934.0,2.474391e+16


In [103]:
bean = df[df['token_name']== 'bean']
ur_bean = df[df['token_name']== 'ur_bean']
bean_3crv = df[df['token_name']== 'bean_3crv']
ur_bean_3crv = df[df['token_name']== 'ur_bean_3crv']

## Deposits by users

As of May 9th there are no deposits for ur_bean, ur_bean_crv3 or bean, so only the mean_crv3 deposits are visualized.

In [114]:
pie_chart(bean_3crv, 4*10**15,"Bean_3crv")
pie_chart(ur_bean,400000,'Ur_bean')
pie_chart(ur_bean_3crv, 500000,"Ur_bean_3crv")
pie_chart(bean,10000,'Bean')

